In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 7]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
M55_path = 'C:\\Users\\marti\\Desktop\\school work\\Dartmouth\\GC_ages\\M55'
M92_path = 'C:\\Users\\marti\\Desktop\\school work\\Dartmouth\\GC_ages\\M92'

In [ ]:
df = pd.read_csv("{}\\M55.dat".format(M55_path),sep='\s+')

In [ ]:
plt.scatter(df['x'],df['y'],s=0.1);

In [ ]:
#determin center
x_guess,y_guess = 3000,3000
radius = 200
def centeroidnp(df):
    fit_stars = df[((df['x']-x_guess)**2 + (df['y']-y_guess)**2) < radius**2]
    length = len(fit_stars)
    x = fit_stars['x'].values
    y = fit_stars['y'].values
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    return sum_x/length, sum_y/length
x_cen, y_cen = centeroidnp(df)
print("X Center is: {}".format(x_cen))
print("Y Center is: {}".format(y_cen))

In [ ]:
#determine MSTO and SGB
plt.scatter(df['VIvega'],df['Vvega'],s=0.1,alpha=0.5,c='grey')
plt.xlim(0.3,0.8)
plt.ylim(16,19)
plt.scatter(0.54,18.0,s=0.7,c='r',label='MSTO')
plt.scatter(0.54+0.05,17.28,s=0.7,c='b',label='SGB')
plt.legend()
plt.gca().invert_yaxis()

$$\textup{V}_{\textup{MSTO}} = 18.0$$
$$\textup{VI}_{\textup{MSTO}} = 0.54$$
$$\textup{V}_{\textup{SGB}} = 17.28$$
$$\textup{VI}_{\textup{SGB}} = 0.59$$

In [ ]:
#Selected ridgeline points
M55_ridgeline = pd.read_csv("{}\\M55_ridgeline_points.csv".format(M55_path),names=['Vvega','Ivega'])

In [ ]:
df_first_cut = df[(df['Vvega'] < 19.78) & (df['Vvega'] > 14.78)]
xp = M55_ridgeline['Vvega'].values
fp = M55_ridgeline['Ivega'].values
plt.scatter(df_first_cut['Vvega'],df_first_cut['Ivega'],s=0.1)
plt.xlabel('Vvega')
plt.ylabel('Ivega')
plt.plot(xp,fp,c='r')


In [ ]:
Vvega = df_first_cut['Vvega'].values
Ivega = df_first_cut['Ivega'].values
VIvega = df_first_cut['VIvega'].values
d = []
for i in range(len(Vvega)):
    x = Vvega[i]
    ivega_needed = np.interp(x,xp,fp) #new linear interpolation
    #use slightly bigger cut to avoid deleting too much points. A finer cut will be applied once the fiducial cmd is generated
    if (((ivega_needed + .1) > Ivega[i]) & (Ivega[i] > (ivega_needed - .1))):
        d.append(df_first_cut.iloc[i][['Vvega','err','Ivega','err.2','VIvega','err.1','x','y']].values.tolist())
M55_second_cut = pd.DataFrame(d,columns=['v','v_err','i','i_err','vi','vi_err','x','y'])

In [ ]:
#check for range of error
plt.scatter(M55_second_cut['v'],M55_second_cut['v_err'],s=0.1)
plt.ylim(0,0.04);

In [ ]:
plt.scatter(M55_second_cut['i'],M55_second_cut['i_err'],s=0.1)
plt.ylim(0,0.04);

In [ ]:
#generate M55 data used to generate fiducial isochrone
V_cut = 17.0
I_cut = 16.2
V_high_err = 0.035
V_low_err = 0.10
I_high_err = 0.035
I_low_err = 0.09
Pass_V = M55_second_cut[((M55_second_cut['v'] <= V_cut) & (M55_second_cut['v_err'] < V_low_err)) | ((M55_second_cut['v'] > V_cut) & (M55_second_cut['v_err'] < V_high_err))]
M55_fitstars_cmd = Pass_V[((Pass_V['i'] <= I_cut) & (Pass_V['i_err'] < I_low_err)) | ((Pass_V['i'] > I_cut) & (Pass_V['i_err'] < I_high_err))]

In [ ]:
plt.scatter(M55_second_cut['vi'],M55_second_cut['v'],s=0.1,c='r',alpha=0.8)
plt.scatter(M55_fitstars_cmd['vi'],M55_fitstars_cmd['v'],s=0.1,c='b')

In [ ]:
M55_fitstars_cmd.to_csv("{}\\M55_fitstars_cmd.dat".format(M55_path),sep=',',index=False)

In [ ]:
print("V_max for firstars is: {}".format(max(M55_fitstars['v'].values)))
print("V_min for firstars is: {}".format(min(M55_fitstars['v'].values)))
print("VI_max for firstars is: {}".format(max(M55_fitstars['vi'].values)))
print("VI_min for firstars is: {}".format(min(M55_fitstars['vi'].values)))